In [64]:
import pandas as pd

In [65]:
bike_realtime_df = pd.read_csv('./bike_realtime0416-0519.csv')
mrt_history_df = pd.read_csv('./mrt_history_groupby03-04.csv')
time_table_df = pd.read_csv('./time_table.csv')
youbike_mrt_distance_df = pd.read_csv('./youbike_mrt_distance.csv')
bike_station_df = pd.read_csv('./bike_station.csv')
mrt_station_info_df = pd.read_csv('./mrt_station_info.csv')
bike_realtime_df.drop(columns="create_time",inplace=True)

In [66]:
bike_realtime_df["source_time"] = pd.to_datetime(bike_realtime_df["source_time"]) 
bike_realtime_df["month"] = bike_realtime_df["source_time"].dt.month
bike_realtime_df["date"] = bike_realtime_df["source_time"].dt.date
bike_realtime_df["hour"] = bike_realtime_df["source_time"].dt.hour
bike_realtime_df["mins"] = bike_realtime_df["source_time"].dt.minute

In [67]:
bike_realtime_df["date"] = pd.to_datetime(bike_realtime_df["date"])
time_table_df["date"] = pd.to_datetime(time_table_df["date"])
mrt_history_df["date"] = pd.to_datetime(mrt_history_df["date"])

In [68]:
start_date = '2024-04-16'
end_date = '2024-04-30'

mrt_history_df = mrt_history_df[(mrt_history_df["date"] >= start_date) & (mrt_history_df["date"] <= end_date)]
bike_realtime_df = bike_realtime_df[(bike_realtime_df["date"] >= start_date) & (bike_realtime_df["date"] <= end_date)]

In [69]:
# bike_realtime_df = bike_realtime_df.loc[(bike_realtime_df["month"]==4)&((bike_realtime_df["hour"]>5) |(bike_realtime_df["hour"]==0)),].reset_index(drop=True)

In [70]:
mrt_history_df["month"] = mrt_history_df["date"].dt.month
mrt_history_df = mrt_history_df.loc[(mrt_history_df["month"]==4)&((mrt_history_df["hour"]>5) |(mrt_history_df["hour"]==0)),].reset_index(drop=True)

In [71]:
aval_bike_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_bike"].mean()).reset_index(drop=False)
aval_bike_mean.rename(columns={"aval_bike":"aval_bike_mean"},inplace=True)
aval_space_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_space"].mean()).reset_index(drop=False)
aval_space_mean.rename(columns={"aval_space":"aval_space_mean"},inplace=True)
df_bike_hour_grby = aval_bike_mean.merge(aval_space_mean,
                     how="left",
                     left_on=["bike_station_id","date","hour"],
                     right_on=["bike_station_id","date","hour"])

In [72]:
df  = df_bike_hour_grby.merge(bike_station_df.loc[:,["bike_station_id","total_space","lat","lng"]],
                              how="left",
                              on="bike_station_id")

In [73]:
df = df.merge(time_table_df.loc[:,["date","day_of_week","day_of_week_name"]],
            on="date",
            how="left")


In [74]:
idx = youbike_mrt_distance_df.groupby("bike_station_id")["distance"].idxmin()
youbike_min_distance_table = youbike_mrt_distance_df.loc[idx,].reset_index(drop=True)

In [75]:
mrt_station_table = youbike_min_distance_table.merge(mrt_station_info_df.loc[:,["mrt_station_id","station_name"]],
                                 on="mrt_station_id",
                                 how="left")
mrt_station_table

,bike_station_id,mrt_station_id,distance,station_name
0,500101001,BR08,0.015941,科技大樓
1,500101002,BR08,0.053192,科技大樓
2,500101003,BR08,0.302305,科技大樓
3,500101004,BR08,0.296290,科技大樓
4,500101005,BR08,0.510676,科技大樓
...,...,...,...,...
1410,500119087,G07,0.645005,公館
1411,500119088,BR08,0.683471,科技大樓
1412,500119089,G07,0.884494,公館
1413,500119090,G07,0.719453,公館


In [76]:
df = df.merge(mrt_station_table,on="bike_station_id",how="left")
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,station_name
0,500101001,2024-04-16,22,7.833333,20.166667,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓
1,500101001,2024-04-16,23,23.000000,5.000000,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓
2,500101001,2024-04-17,0,20.500000,7.500000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
3,500101001,2024-04-17,1,16.666667,11.333333,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
4,500101001,2024-04-17,2,18.000000,10.000000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438546,500119091,2024-04-29,16,11.500000,6.500000,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
438547,500119091,2024-04-29,17,9.000000,9.000000,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
438548,500119091,2024-04-29,18,0.500000,17.500000,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
438549,500119091,2024-04-29,19,6.666667,11.333333,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓


In [77]:
df.dtypes

bike_station_id              int64
date                datetime64[ns]
hour                         int32
aval_bike_mean             float64
aval_space_mean            float64
total_space                  int64
lat                        float64
lng                        float64
day_of_week                  int64
day_of_week_name            object
mrt_station_id              object
distance                   float64
station_name                object
dtype: object

In [78]:
mrt_history_df

,date,hour,mrt_station,enter_num,exit_num,month
0,2024-04-16,0,龍山寺,127,142.0,4
1,2024-04-16,0,麟光,10,30.0,4
2,2024-04-16,0,頭前庄,15,35.0,4
3,2024-04-16,0,頂溪,71,230.0,4
4,2024-04-16,0,頂埔,8,104.0,4
...,...,...,...,...,...,...
33625,2024-04-30,23,三重國小,116,423.0,4
33626,2024-04-30,23,三重,87,183.0,4
33627,2024-04-30,23,三民高中,89,453.0,4
33628,2024-04-30,23,三和國中,99,356.0,4


In [79]:
df = df.merge(mrt_history_df,left_on=["station_name","date","hour"],right_on=["mrt_station","date","hour"],how="left")

In [80]:
df["bike_rate"] = df["aval_bike_mean"]/df["total_space"]

In [81]:
df["total_visitors"] = df["enter_num"] + df["exit_num"]
df["ex_enter_diff"] =  df["exit_num"] - df["enter_num"] 

In [82]:
df = df.loc[~df["hour"].isin([22,23,0,1,2,3,4,5]),]

In [83]:
df.loc[df["mrt_station"].isna(),"mrt_station_id"].unique()

array(['O12'], dtype=object)

In [89]:
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,station_name,mrt_station,enter_num,exit_num,month,bike_rate,total_visitors,ex_enter_diff
8,500101001,2024-04-17,6,23.500000,4.500000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,科技大樓,152.0,193.0,4.0,0.839286,345.0,41.0
9,500101001,2024-04-17,7,17.833333,10.166667,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,科技大樓,741.0,938.0,4.0,0.636905,1679.0,197.0
10,500101001,2024-04-17,8,3.166667,24.833333,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,科技大樓,1774.0,2219.0,4.0,0.113095,3993.0,445.0
11,500101001,2024-04-17,9,0.666667,27.333333,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,科技大樓,982.0,1402.0,4.0,0.023810,2384.0,420.0
12,500101001,2024-04-17,10,0.500000,27.500000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,科技大樓,470.0,678.0,4.0,0.017857,1148.0,208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438546,500119091,2024-04-29,16,11.500000,6.500000,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓,科技大樓,816.0,749.0,4.0,0.638889,1565.0,-67.0
438547,500119091,2024-04-29,17,9.000000,9.000000,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓,科技大樓,1442.0,1152.0,4.0,0.500000,2594.0,-290.0
438548,500119091,2024-04-29,18,0.500000,17.500000,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓,科技大樓,2042.0,1650.0,4.0,0.027778,3692.0,-392.0
438549,500119091,2024-04-29,19,6.666667,11.333333,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓,科技大樓,1294.0,898.0,4.0,0.370370,2192.0,-396.0


In [84]:
bk_ids = list(df["bike_station_id"].unique())
corr_df = pd.DataFrame(columns=["bike_station_id","corr_enter_bike","corr_exit_bike","corr_total_visitors","corr_ex_enter_diff","corr_enter_exit"])

In [85]:
for bk_id in bk_ids:
    tem = df.loc[df["bike_station_id"]==bk_id,]
    cor_matrix = tem.loc[:,["bike_rate","enter_num","total_visitors","ex_enter_diff","exit_num"]].corr()
    corr_df_tem = pd.DataFrame({
        "bike_station_id":[bk_id],
        "corr_enter_bike":[cor_matrix.loc["enter_num","bike_rate"]],
        "corr_exit_bike":cor_matrix.loc["exit_num","bike_rate"],
        "corr_total_visitors":cor_matrix.loc["total_visitors","bike_rate"],
        "corr_ex_enter_diff":cor_matrix.loc["ex_enter_diff","bike_rate"],
        "corr_enter_exit":cor_matrix.loc["exit_num","enter_num"]
    })
    corr_df = pd.concat([corr_df,corr_df_tem],axis=0)

C:\Users\andy\AppData\Local\Temp\ipykernel_7020\3527425555.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  corr_df = pd.concat([corr_df,corr_df_tem],axis=0)


In [87]:
corr_df.sort_values(["corr_ex_enter_diff"],ascending=True,inplace=True)
corr_df

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit
0,500119048,0.557353,-0.031539,0.351141,-0.702179,0.578933
0,500113040,0.106305,-0.483300,-0.243294,-0.673540,0.583025
0,500105005,0.355522,-0.626543,-0.144482,-0.663768,-0.046347
0,500101181,0.466719,-0.063419,0.276839,-0.619741,0.578933
0,500110076,0.554818,-0.163543,0.286596,-0.613494,0.256228
...,...,...,...,...,...,...
0,500109065,NaN,NaN,NaN,NaN,0.828852
0,500110028,NaN,NaN,NaN,NaN,0.034303
0,500111081,NaN,NaN,NaN,NaN,0.749373
0,500111085,NaN,NaN,NaN,NaN,0.665863


In [101]:
corr_df.loc[corr_df["corr_ex_enter_diff"]>0.5,]

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit
0,500101135,-0.420073,0.177642,-0.138985,0.500841,0.306073
0,500110088,-0.410343,0.263458,-0.055550,0.504678,0.154108
0,500101194,-0.436386,0.178332,-0.196928,0.519405,0.256228
0,500106080,-0.131015,0.168566,0.023386,0.523342,0.836240
0,500101091,-0.389076,-0.110845,-0.266265,0.543995,0.851051
0,500101025,-0.497176,-0.053594,-0.351216,0.556257,0.578933
0,500108150,-0.385396,0.312114,-0.025041,0.559205,0.232940
0,500106107,-0.406510,0.398260,0.037887,0.559745,-0.026292
0,500106006,-0.188117,0.132824,-0.030107,0.573014,0.842839
0,500101034,-0.480498,0.111758,-0.263414,0.677898,0.578933


In [103]:
corr_df.loc[corr_df["bike_station_id"]==500105005,]

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit
0,500105005,0.355522,-0.626543,-0.144482,-0.663768,-0.046347


In [90]:
corr_df_merge = corr_df.merge(mrt_station_table,how="left",on="bike_station_id")

In [109]:
bike_corr_high_id = list(corr_df.loc[corr_df["corr_ex_enter_diff"]<=-0.5,"bike_station_id"])
bike_corr_high_id

[500119048,
 500113040,
 500105005,
 500101181,
 500110076,
 500110026,
 500103048,
 500101021,
 500101163,
 500101009,
 500106004,
 500119045,
 500110027]

In [110]:
df_for_scatter_plot = df.loc[df["bike_station_id"].isin(bike_corr_high_id),]

In [112]:
df_for_scatter_plot.to_csv("df_high_cor.csv",index=False,encoding="utf-8-sig")

In [20]:
df.to_csv("test2.csv",encoding="utf-8-sig",index=False)

In [21]:
#############################